In [32]:
def sim(img, text):
    
    filler_img = 0
    for i in img:
        filler_img += i ** 2
    filler_text = 0
    for j in img:
        filler_text += j ** 2

    numer = np.dot(img,text)
    denom = np.sqrt(filler_img) * np.sqrt(filler_text)
    
    return numer/denom

#prob delete latr

In [9]:
import mygrad as mg
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch 
test = [(48, 318556, 388611), (67, 116100, 412616), (126, 318556, 259951)]

In [18]:
def loss_func(triples_list, Wembed):
    text, img_good, img_bad = zip(*triples_list)
    text = np.array(text).reshape(-1, 1)
    img_good = np.array(img_good).reshape(-1, 1)
    img_bad = np.array(img_bad).reshape(-1, 1)
    #getting tuples n turning them into the correct arrays

    img_good_embed = np.dot(img_good, Wembed.detach().numpy())
    img_bad_embed = np.dot(img_bad, Wembed.detach().numpy())

    sim_to_good = cosine_similarity(text, img_good_embed)
    sim_to_bad = cosine_similarity(text, img_bad_embed)
    #sim_to_good = sim(text, img_good)
    #sim_to_bad = sim(text, img_bad)
    #my other ver of cosine sim
    
    y = np.ones(len(text)) #1 or -1, 1 makes it so that its x1 - x2 (vice versa with -1).
    
    return mg.nnet.losses.margin_ranking_loss(sim_to_good, sim_to_bad, y, margin = 0.25) #what notebook said

In [19]:
#Wembed = mg.nnet.initializers.glorot_normal(512,200)
Wembed = torch.nn.Parameter(torch.randn(512, 200), requires_grad=True)
optimizer = torch.optim.SGD([Wembed], lr=0.1, momentum=0.9)

In [20]:
def train(triples_list, Wembed, optimizer, num_epochs, batch_size=32):
    for epoch in range(num_epochs):
        for i in range(0, len(triples_list), batch_size):
            batch = triples_list[i:i+batch_size]
            loss = loss_func(batch, Wembed)  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [21]:
train(test, Wembed, optimizer, num_epochs=1)

ValueError: shapes (3,1) and (512,200) not aligned: 1 (dim 1) != 512 (dim 0)